In [18]:
import pandas as pd
import json
from tqdm import tqdm

import mmcv

In [2]:
df = pd.read_csv("/workspace/data/df_train_study_level_npy640_3_w_bbox_hw.csv")
df.head()

,id,x,y,w,h,study_id,image_id,have_box,dicom_path,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,cv,npy_path,is_none,height,width
0,04f41a8958f7_image,688.06282,966.82563,518.48212,1130.17438,6e4a0581cefe,04f41a8958f7,1,/workspace/data/train/6e4a0581cefe/018ed20fa9c...,0,1,0,0,0,/workspace/data/train_640_2/04f41a8958f7.png,0,2490,3408
1,04f41a8958f7_image,2482.36026,1636.77436,652.47168,652.47168,6e4a0581cefe,04f41a8958f7,1,/workspace/data/train/6e4a0581cefe/018ed20fa9c...,0,1,0,0,0,/workspace/data/train_640_2/04f41a8958f7.png,0,2490,3408
2,04f41a8958f7_image,1235.67308,1628.03597,509.74353,559.26147,6e4a0581cefe,04f41a8958f7,1,/workspace/data/train/6e4a0581cefe/018ed20fa9c...,0,1,0,0,0,/workspace/data/train_640_2/04f41a8958f7.png,0,2490,3408
3,0572ef0d0c1a_image,1818.65264,233.50598,613.04395,839.53784,adbfed2da701,0572ef0d0c1a,1,/workspace/data/train/adbfed2da701/e2fa197720c...,0,1,0,0,3,/workspace/data/train_640_2/0572ef0d0c1a.png,0,2436,3032
4,0572ef0d0c1a_image,598.60492,61.37052,688.54175,881.81674,adbfed2da701,0572ef0d0c1a,1,/workspace/data/train/adbfed2da701/e2fa197720c...,0,1,0,0,3,/workspace/data/train_640_2/0572ef0d0c1a.png,0,2436,3032


https://github.com/open-mmlab/mmdetection/blob/master/docs/2_new_data_model.md

In [20]:
for cv in range(5):
    if cv == 1:
        break
    to_json = [
        [df[df.cv != cv].reset_index(drop=True), f"out.json"],
#         [df[df.cv == cv].reset_index(drop=True), f"../config/val_cv{cv}.json"]
    ]
    
    for df_to_json, out_file in to_json:

        image_id_unique = df_to_json["id"].unique()

        images = []
        annotations = []
        obj_count = 0

        for idx, image_id in tqdm(enumerate(image_id_unique)):
            if idx == 3:
                break
            target_df = df[df["id"] == image_id]

            img_path = target_df.iloc[0:1]["npy_path"].iloc[0]
            filename = img_path.split('/')[-1]
            height, width = mmcv.imread(img_path).shape[:2]

            images.append(dict(
                id=idx,
                file_name=filename,
                height=height,
                width=width))


            for row_id in range(len(target_df)):
                row = target_df.iloc[row_id]
                x_min, y_min, x_max, y_max = (
                    row["x"], row["y"],row["x"] + row["w"], row["y"] + row["h"])


                data_anno = dict(
                    image_id=idx,
                    id=obj_count,
                    category_id=0,
                    bbox=[x_min, y_min, x_max - x_min, y_max - y_min],
                    area=(x_max - x_min) * (y_max - y_min),
                    iscrowd=0)
                annotations.append(data_anno)
                obj_count += 1

        coco_format_json = dict(
            images=images,
            annotations=annotations,
            categories=[{'id':0, 'name': 'opacity'}])
        mmcv.dump(coco_format_json, out_file)
        break

3it [00:00, 17.81it/s]


In [21]:
for cv in range(5):
    to_json = [
        [df[df.cv != cv].reset_index(drop=True), f"../config/train_cv{cv}.json"],
        [df[df.cv == cv].reset_index(drop=True), f"../config/val_cv{cv}.json"]
    ]
    
    for df_to_json, out_file in to_json:

        image_id_unique = df_to_json["id"].unique()

        images = []
        annotations = []
        obj_count = 0

        for idx, image_id in tqdm(enumerate(image_id_unique)):
            target_df = df[df["id"] == image_id]

            img_path = target_df.iloc[0:1]["npy_path"].iloc[0]
            filename = img_path.split('/')[-1]
            height, width = mmcv.imread(img_path).shape[:2]

            images.append(dict(
                id=idx,
                file_name=filename,
                height=height,
                width=width))


            for row_id in range(len(target_df)):
                row = target_df.iloc[row_id]
                x_min, y_min, x_max, y_max = (
                    row["x"], row["y"],row["x"] + row["w"], row["y"] + row["h"])


                data_anno = dict(
                    image_id=idx,
                    id=obj_count,
                    category_id=0,
                    bbox=[x_min, y_min, x_max - x_min, y_max - y_min],
                    area=(x_max - x_min) * (y_max - y_min),
                    iscrowd=0)
                annotations.append(data_anno)
                obj_count += 1

        coco_format_json = dict(
            images=images,
            annotations=annotations,
            categories=[{'id':0, 'name': 'opacity'}])
        mmcv.dump(coco_format_json, out_file)

4894it [05:19, 15.30it/s]
1223it [01:20, 15.23it/s]
4897it [05:20, 15.26it/s]
1220it [01:18, 15.51it/s]
4896it [05:25, 15.05it/s]
1221it [01:19, 15.37it/s]
4889it [05:47, 14.06it/s]
1228it [01:40, 12.24it/s]
4892it [06:35, 12.38it/s]
1225it [01:21, 15.03it/s]


In [13]:
len(annotations)

1919

In [14]:
len(images)

1225

In [17]:
len(df.id.unique())

6117